## OPENAI API - Testing    
##  Kelly Anderson 05/26/2023

## In this test example, I am reading in two pdf files from car rentals that I have been sent from previous trips.  
## The files are extremely different in their layout and contents, but with the help of a PDF reader library, and OpenAI API, I am able to extract some common data elements, and am able to consolidate the data

## This is a small example, but using similar logic, we could easily extract data from large amounts of different and unstructured data.   Very exciting!!

###  The steps taken in this work is as follows:
### 1. Load pdf files from trip1 - Maui - then pass this into OpenAI, prompting for fields that we are looking for
### 2. Load pdf files from trip2 - MSP - then pass this into OpenAI, prompting for fields that we are looking for
### 3. Consolidate the data into a dataframe that could then be used for loading into a database or data warehouse.


In [7]:
#!pip install PyPDF2
import PyPDF2

import openai
import os

import pandas as pd

In [2]:
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
#        pdf_reader = PyPDF2.PdfFileReader(file)
        pdf_reader = PyPDF2.PdfReader(file)
#        num_pages = pdf_reader.numPages
        num_pages = len(pdf_reader.pages) 
        print(num_pages)
        extracted_text = ''

        for page_number in range(num_pages):
#            page = pdf_reader.getPage(page_number)
#reader.getPage(pageNumber) is deprecated and was removed in PyPDF2 3.0.0. Use reader.pages[page_number] instead.
            page = pdf_reader.pages[page_number]
            print(page.extract_text())
#            extracted_text += page.extractText()
#extractText is deprecated and was removed in PyPDF2 3.0.0. Use extract_text instead.
            extracted_text += page.extract_text()
            
        return extracted_text



In [3]:
# Provide the path to your PDF file
pdf_file_path = 'DEFAULTRESERVATIONFORM.PDF'
#pdf_file_path = '358_Osgood_Details.pdf'
extracted_text = extract_text_from_pdf(pdf_file_path)
print(extracted_text)

2
LESSOR:
RENTER:
ALLSAVE / FOX AIRPORT 
LOCATION MAUI
81 ALAE ST.
KAHULUI MAUI AIRPORT, HI 
96732
808-868-0071
ANDERSON, KELLY
 
ANDERSKD@HOTMAIL.COM
CONFIRM. NO.:
BOOKED DATE:
WFX060527E
12/11/2022
RENTAL
VEHICLE
Pick-up date
CFAR / COMPACT SUV
Unit type/description
04/06/2023 12:00 PM
Return date
Total Days
04/12/2023 11:00 AM
6
Agency code
Total 
0
Unit
Description
Total
390.00
6 Day(s) @65.00
0.00
0 Charged (s) @0.00
16.26
STATE TAX (4.17%)
33.00
HIGHWAY TAX (5.50)
21.48
LICENSE FEE (3.58)
21.30
A/S SURCHARGE (3.55)
Total Charges
482.04
0.00
Total Payments
Balance
482.04
ALLSAVE CAR RENTAL - FOX - NU Reservation Summary

Thank you for choosing Allsave Car Rental/Fox Rent a Car Airport/Kahului Location.
Shuttle Info:       Please DO NOT! take the train across from baggage claim for Shuttle
Our Airport shuttle pickup is around the corner from baggage claim and runs daily 
every 15-20 minutes during business hours.
When you exit outside baggage claim walk to your right just past the 

In [4]:
len(extracted_text)

1854

In [5]:
extracted_text_hi = extracted_text

In [8]:

# 3. Load the key from a file
with open('key.txt', 'r') as f:
    api_key = f.read().strip('\n')
    assert api_key.startswith('sk-'), 'Error loading the API key. The API key starts with "sk-"' 
openai.api_key = api_key

In [10]:
#prompt = 'Can you find the lease start and end dates for the following lease:' + extracted_text
prompt = '''Can you find the 
car rental company, 
car rentee, 
address of pickup, 
rental start,
rental end date, 
rent amount,
total balance due, 
type of car, 
reservation number,
and deposit for the following car rental agreement:''' +  extracted_text_hi

messages = [
    {'role': 'system', 'content': 'Answer as concisely as possible with the exact field names'},
    {'role': 'user', 'content': prompt}
]

response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = messages,
    temperature = 0.1,
    max_tokens = 1000,
    #stop=[',','-']
    top_p=.1
)
r = response.choices[0]['message']['content']
print(r)
#top_p set to .1

Car rental company: Allsave Car Rental/Fox Rent a Car Airport/Kahului Location
Car rentee: Anderson, Kelly
Address of pickup: 81 Alae St., Kahului Maui Airport, HI 96732
Rental start date: 04/06/2023 12:00 PM
Rental end date: 04/12/2023 11:00 AM
Rent amount: $390.00
Total balance due: $482.04
Type of car: CFAR/Compact SUV
Reservation number: WFX060527E
Deposit: $150.00 (for credit card users) or $300.00 to $500.00 (for debit card or cash users)


##  The results from OpenAI are pretty well formatted for us to convert to a pandas dataframe

In [11]:
text = r

data = {
    'Car rental company': [],
    'Car rentee': [],
    'Address of pickup': [],
    'Rental start date': [],
    'Rental end date': [],
    'Rent amount': [],
    'Total balance due': [],
    'Type of car': [],
    'Reservation number': [],
    'Deposit': []
}

lines = text.strip().split('\n')
for line in lines:
    header, value = line.split(': ')
    data[header.strip()].append(value.strip())

df_hi = pd.DataFrame(data)

print(df_hi)

                                  Car rental company       Car rentee  \
0  Allsave Car Rental/Fox Rent a Car Airport/Kahu...  Anderson, Kelly   

                             Address of pickup    Rental start date  \
0  81 Alae St., Kahului Maui Airport, HI 96732  04/06/2023 12:00 PM   

       Rental end date Rent amount Total balance due       Type of car  \
0  04/12/2023 11:00 AM     $390.00           $482.04  CFAR/Compact SUV   

  Reservation number                                            Deposit  
0         WFX060527E  $150.00 (for credit card users) or $300.00 to ...  


In [13]:
df_hi.head()

,Car rental company,Car rentee,Address of pickup,Rental start date,Rental end date,Rent amount,Total balance due,Type of car,Reservation number,Deposit
0,Allsave Car Rental/Fox Rent a Car Airport/Kahu...,"Anderson, Kelly","81 Alae St., Kahului Maui Airport, HI 96732",04/06/2023 12:00 PM,04/12/2023 11:00 AM,$390.00,$482.04,CFAR/Compact SUV,WFX060527E,$150.00 (for credit card users) or $300.00 to ...


In [14]:
##Here is the full response - for review

In [15]:
response

<OpenAIObject chat.completion id=chatcmpl-7KXKpnjq3l0LzuOGAMY18IPMmwpD1 at 0x1c8dc03b650> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Car rental company: Allsave Car Rental/Fox Rent a Car Airport/Kahului Location\nCar rentee: Anderson, Kelly\nAddress of pickup: 81 Alae St., Kahului Maui Airport, HI 96732\nRental start date: 04/06/2023 12:00 PM\nRental end date: 04/12/2023 11:00 AM\nRent amount: $390.00\nTotal balance due: $482.04\nType of car: CFAR/Compact SUV\nReservation number: WFX060527E\nDeposit: $150.00 (for credit card users) or $300.00 to $500.00 (for debit card or cash users)",
        "role": "assistant"
      }
    }
  ],
  "created": 1685129055,
  "id": "chatcmpl-7KXKpnjq3l0LzuOGAMY18IPMmwpD1",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 149,
    "prompt_tokens": 637,
    "total_tokens": 786
  }
}

# ADVANTAGE - MSP - Rental Agreement Receipt - MSP-147615.pdf

In [16]:
# Provide the path to your PDF file
pdf_file_path = 'ADVANTAGE - MSP - Rental Agreement Receipt - MSP-147615.pdf'
#pdf_file_path = '358_Osgood_Details.pdf'
extracted_text = extract_text_from_pdf(pdf_file_path)
print(extracted_text)

2
RENTAL RECORD Rental Record No.
DATE TIME
IN
DATE TIME
OUT
VEHICLE NO. FUEL OUT
VEHICLE LIC. NO.
VEH. MAKE - BODY STYLE
VEHICLE TO BE RETURNED TO (CITY/STATE) LOC. NO.
VEHICLE RETURNED AT (CITY/STATE) AREA & LOCATION NO.
All daily charges based on 24-hour rental dayVEH.CLS.FUEL INMILEAGE
DRIVENMILEAGE
OUTMILEAGE
IN
TO BE PAID BY
HOME/BUSINESS ADDRESS HOME/BUSINESS PHONE NO.
CITY/STATE/COUNTRY ZIP CODE
EMAIL ADDRESS
DRIVER’S LICENSE NO.     STATE/COUNTRY      EXPIRES D.O.B.
Additional Driver Dr. Lic. # State Exp. Date         Date of Birth
Address City State Zip
You are responsible for payment of all tolls. This Vehicle is equipped with the ability 
to pay tolls electronically, and we offer EZTOL, a fee service that permits you to use automated 
express lanes within the state of Minnesota  without paying a toll at the time you drive through 
the express lane.  
This is a non-smoking vehicle. If the vehicle is returned smelling of smoke from any source, you 
will pay us a cleaning fee 

RENTAL RECORD Rental Record No.
DATE TIME
IN
DATE TIME
OUT
VEHICLE NO. FUEL OUT
VEHICLE LIC. NO.
VEH. MAKE - BODY STYLE
VEHICLE TO BE RETURNED TO (CITY/STATE) LOC. NO.
VEHICLE RETURNED AT (CITY/STATE) AREA & LOCATION NO.
All daily charges based on 24-hour rental dayVEH.CLS.FUEL INMILEAGE
DRIVENMILEAGE
OUTMILEAGE
IN
TO BE PAID BY
HOME/BUSINESS ADDRESS HOME/BUSINESS PHONE NO.
CITY/STATE/COUNTRY ZIP CODE
EMAIL ADDRESS
DRIVER’S LICENSE NO.     STATE/COUNTRY      EXPIRES D.O.B.
Additional Driver Dr. Lic. # State Exp. Date         Date of Birth
Address City State Zip
You are responsible for payment of all tolls. This Vehicle is equipped with the ability 
to pay tolls electronically, and we offer EZTOL, a fee service that permits you to use automated 
express lanes within the state of Minnesota  without paying a toll at the time you drive through 
the express lane.  
This is a non-smoking vehicle. If the vehicle is returned smelling of smoke from any source, you 
will pay us a cleaning fee ac

In [17]:
len(extracted_text)

25704

In [18]:
# Due to the size of this file, we need to just take the first chunk of 6000 characters.  
# If needed though, we could loop through each chunk to find all of the values we are looking for

In [19]:
def extract_substring(string, start, end):
    return string[start:end+1]

# Example usage
substring = extract_substring(extracted_text, 0, 6000)
print(substring)

RENTAL RECORD Rental Record No.
DATE TIME
IN
DATE TIME
OUT
VEHICLE NO. FUEL OUT
VEHICLE LIC. NO.
VEH. MAKE - BODY STYLE
VEHICLE TO BE RETURNED TO (CITY/STATE) LOC. NO.
VEHICLE RETURNED AT (CITY/STATE) AREA & LOCATION NO.
All daily charges based on 24-hour rental dayVEH.CLS.FUEL INMILEAGE
DRIVENMILEAGE
OUTMILEAGE
IN
TO BE PAID BY
HOME/BUSINESS ADDRESS HOME/BUSINESS PHONE NO.
CITY/STATE/COUNTRY ZIP CODE
EMAIL ADDRESS
DRIVER’S LICENSE NO.     STATE/COUNTRY      EXPIRES D.O.B.
Additional Driver Dr. Lic. # State Exp. Date         Date of Birth
Address City State Zip
You are responsible for payment of all tolls. This Vehicle is equipped with the ability 
to pay tolls electronically, and we offer EZTOL, a fee service that permits you to use automated 
express lanes within the state of Minnesota  without paying a toll at the time you drive through 
the express lane.  
This is a non-smoking vehicle. If the vehicle is returned smelling of smoke from any source, you 
will pay us a cleaning fee ac

In [20]:
#prompt = 'Can you find the lease start and end dates for the following lease:' + extracted_text
prompt = '''Can you find the 
car rental company, 
car rentee, 
address of car rental, 
rental start,
rental end date, 
rent amount,
total balance due, 
type of car, 
reservation number,
and deposit for the following car rental agreement:''' +  substring

messages = [
    {'role': 'system', 'content': 'Answer as concisely as possible with the exact field names'},
    {'role': 'user', 'content': prompt}
]

response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = messages,
    temperature = 0.1,
    max_tokens = 1000,
    #stop=[',','-']
    top_p=.1
)
r = response.choices[0]['message']['content']
print(r)
#top_p set to .1

Car rental company: Advantage - MSP
Car rentee: Kelly Anderson
Address of car rental: 4650 Glumack Dr, St. Paul, MN 55111
Rental start date: 12/22/2017
Rental end date: 12/30/2017
Rent amount: $409.77
Total balance due: $0.00
Type of car: Hyundai Santa Fe Sport 4WD/R
Reservation number: MSP-147615
Deposit: Not specified in the agreement.


In [24]:
text = r

data = {
    'Car rental company': [],
    'Car rentee': [],
    'Address of car rental': [],
    'Rental start date': [],
    'Rental end date': [],
    'Rent amount': [],
    'Total balance due': [],
    'Type of car': [],
    'Reservation number': [],
    'Deposit': []
}

lines = text.strip().split('\n')
for line in lines:
    header, value = line.split(': ')
    data[header.strip()].append(value.strip())

df_mn = pd.DataFrame(data)

print(df_mn)

  Car rental company      Car rentee                Address of car rental  \
0    Advantage - MSP  Kelly Anderson  4650 Glumack Dr, St. Paul, MN 55111   

  Rental start date Rental end date Rent amount Total balance due  \
0        12/22/2017      12/30/2017     $409.77             $0.00   

                    Type of car Reservation number  \
0  Hyundai Santa Fe Sport 4WD/R         MSP-147615   

                           Deposit  
0  Not specified in the agreement.  


In [25]:
df_consolidated = pd.concat([df_hi, df_mn], ignore_index=True)

In [26]:
df_consolidated[['Car rental company', 'Car rentee', 'Rental start date', 'Rental end date', 'Rent amount', 'Type of car']]

,Car rental company,Car rentee,Rental start date,Rental end date,Rent amount,Type of car
0,Allsave Car Rental/Fox Rent a Car Airport/Kahu...,"Anderson, Kelly",04/06/2023 12:00 PM,04/12/2023 11:00 AM,$390.00,CFAR/Compact SUV
1,Advantage - MSP,Kelly Anderson,12/22/2017,12/30/2017,$409.77,Hyundai Santa Fe Sport 4WD/R
